# [Create Agent with Azure AI Search](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview#create-agent-with-azure-ai-search)
Azure AI Search is an enterprise search system for high-performance applications. It integrates with Azure OpenAI Service and Azure Machine Learning, offering advanced search technologies like vector search and full-text search. Ideal for knowledge base insights, information discovery, and automation.<br/><br/>
**IMPORTANT**: in order to create the index leveraging the "Import and Vectorize" wizard, the `Storage Blob Data Contributor` role on the Storage Account is needed for the Azure AI Managed Identity.

## Create a connection to Azure AI Search using CLI

1. First of all, let's check which connections we have, associated to our project mmai-hub04-prj01-fvye:
   ```az ml connection list --resource-group mmai04-rg --workspace-name mmai-hub04-prj01-fvye```
3. Now, create a new yaml file with the configuration for Azure AI Search, using key-based or key-less (as in this case) authentication. **Please note that the "metadata" section must be filled as shown, including the ResourceId that must contain the connection name reporten on line 1**:
```
name: mmai-hub04-fvye-connection-AISearch
type: azure_ai_search
endpoint: https://mmai-hub04-ai-search-fvye.search.windows.net/
is_shared: true
metadata:
  ApiType: Azure
  ResourceId: /subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.Search/searchServices/mmai-hub04-fvye-connection-AISearch
  type: azure_ai_search
  ```

3. Run the command `az ml connection create --file aisearchconnection.yml --resource-group mmai04-rg --workspace-name mmai-hub04-prj01-fvye`


# Constants

In [82]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv(dotenv_path='../infra/credentials.env', override=True)
 
model_name = os.environ["MODEL_DEPLOYMENT_NAME"]
project_connection_string = os.environ["AI_PROJECT_CONNECTION_STRING"]

print(f'Project Connection String: <...{project_connection_string[-30:]}>')

Project Connection String: <...ing-rg;aifoundry-upskilling-pj>


# Create AI Foundry Project Client

In [98]:
from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool# <<<<<<<<<<<<<<< SPECIFIC FOR AZURE AI SEARCH
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

project_client.scope

ModuleNotFoundError: No module named 'aiohttp'

# List existing connections and highlight the "AI Search" ones

In [96]:
conn_list = project_client.connections.list()
conn_id = ""
for conn in conn_list:    
    if conn.connection_type == "CognitiveSearch":
        print(f">>>>>>> AI SEARCH FOUND: {conn}")
        conn_id = conn.id
    else:
        print(conn)

>>>>>>> AI SEARCH FOUND: {
 "name": "AzureAISearch",
 "id": "/subscriptions/dbc342d5-96b5-4aef-a49d-5f6cbd7db6ce/resourceGroups/aifoundry-upskilling-rg/providers/Microsoft.MachineLearningServices/workspaces/aifoundry-upskilling-pj/connections/AzureAISearch",
 "authentication_type": "ApiKey",
 "connection_type": "CognitiveSearch",
 "endpoint_url": "https://ai-search-abutneva687267079310.search.windows.net",
 "key": null
 "token_credential": null
}

{
 "name": "ai-aifoundryupskillinghub687267079310_aoai",
 "id": "/subscriptions/dbc342d5-96b5-4aef-a49d-5f6cbd7db6ce/resourceGroups/aifoundry-upskilling-rg/providers/Microsoft.MachineLearningServices/workspaces/aifoundry-upskilling-pj/connections/ai-aifoundryupskillinghub687267079310_aoai",
 "authentication_type": "ApiKey",
 "connection_type": "AzureOpenAI",
 "endpoint_url": "https://ai-aifoundryupskillinghub687267079310.openai.azure.com",
 "key": null
 "token_credential": null
}

{
 "name": "ai-aifoundryupskillinghub687267079310",
 "id": "/s

# Chosen AI Search Connection

In [77]:
conn_id

'/subscriptions/dbc342d5-96b5-4aef-a49d-5f6cbd7db6ce/resourceGroups/aifoundry-upskilling-rg/providers/Microsoft.MachineLearningServices/workspaces/aifoundry-upskilling-pj/connections/AzureAISearch'

# Initialize `AzureAISearchTool`, adding the connection id to it

In [97]:
ai_search = AzureAISearchTool(index_connection_id=conn_id, 
                              index_name=os.environ["AZURE_SEARCH_INDEX_NAME"],
                              query_type=AzureAISearchQueryType.SIMPLE)
print(f"ai_search.definitions: {ai_search.definitions}")
print(f"ai_search.resources: {ai_search.resources}")

NameError: name 'AzureAISearchQueryType' is not defined

# Create AI Foundry Agent

In [68]:
# Create agent with AI search tool and process assistant run
agent = project_client.agents.create_agent(
    model=model_name,
    name="aisearch-agent",
    instructions="You are a helpful assistant",
    tools=ai_search.definitions,
    tool_resources=ai_search.resources,
    headers={"x-ms-enable-preview": "true"},
)

agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_mOHtQ7GuAUMihaVWi5cMJloc', 'object': 'assistant', 'created_at': 1744302104, 'name': 'aisearch-agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'azure_ai_search'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'azure_ai_search': {'indexes': [{'index_connection_id': '/subscriptions/dbc342d5-96b5-4aef-a49d-5f6cbd7db6ce/resourceGroups/aifoundry-upskilling-rg/providers/Microsoft.MachineLearningServices/workspaces/aifoundry-upskilling-pj/connections/AzureAISearch', 'index_name': 'my-index-2025-04-10-2025-04-10-sdk', 'query_type': 'vector_semantic_hybrid', 'top_k': 5, 'filter': None}]}}, 'metadata': {}, 'response_format': 'auto'}>

# Create the thread and attach a new message to it

In [69]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id, 
    role="user", 
    content="What was the country with the lowest GDP in 2023? Do **NOT** use your own internal knowledge.",
)
print(f"Created message: {message}")

Created thread: {'id': 'thread_VCyyubsqIqRjlcQgPYwTp8zp', 'object': 'thread', 'created_at': 1744302109, 'metadata': {}, 'tool_resources': {}}

Created message: {'id': 'msg_Zg33YSLbVdSw9cSwIFuYcBNl', 'object': 'thread.message', 'created_at': 1744302109, 'assistant_id': None, 'thread_id': 'thread_VCyyubsqIqRjlcQgPYwTp8zp', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What was the country with the lowest GDP in 2023? Do **NOT** use your own internal knowledge.', 'annotations': []}}], 'attachments': [], 'metadata': {}}


# Run the agent synchronously

In [70]:
%%time
# Create and process assistant run in thread with tools
run = project_client.agents.create_and_process_run\
    (thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}.\n\nRun: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: failed.

Run: {'id': 'run_aUGZnSfeXioREnFCuX5aOpwr', 'object': 'thread.run', 'created_at': 1744302115, 'assistant_id': 'asst_mOHtQ7GuAUMihaVWi5cMJloc', 'thread_id': 'thread_VCyyubsqIqRjlcQgPYwTp8zp', 'status': 'failed', 'started_at': 1744302116, 'expires_at': None, 'cancelled_at': None, 'failed_at': 1744302117, 'completed_at': None, 'required_action': None, 'last_error': {'code': 'tool_user_error', 'message': 'Error: invalid_search_request; Invalid configuration for Azure AI Search index. Query type vector_semantic_hybrid requires a vector field with integrated vectorizer, but none was found'}, 'model': 'gpt-4o', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'azure_ai_search'}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 339, 'completion_

# Fetch messages from the thread after the agent run execution

In [19]:
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent

if run.status == 'completed':    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    messages_nr = len(messages.data)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages.data), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageImageFileContent):
                print(f"\nCONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                print(f"\nCONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages:


===== MESSAGE 1 =====

CONTENT 1 (MessageTextContent) --> Text: What was the country with the lowest GDP in 2023? Do **NOT** use your own internal knowledge.

===== MESSAGE 2 =====

CONTENT 1 (MessageTextContent) --> Text: Based on the documents retrieved, there is no specific information about the country with the lowest GDP in 2023. Therefore, I cannot provide an answer based on the given documents.


In [20]:
from azure.ai.projects.models import FilePurpose, MessageRole

# Get the last message from the sender
last_msg = messages.get_last_text_message_by_role(MessageRole.AGENT)
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

Last Message: Based on the documents retrieved, there is no specific information about the country with the lowest GDP in 2023. Therefore, I cannot provide an answer based on the given documents.


# Print annotations from the messages

In [16]:
print(f"Number of annotation(s): {len(last_msg.text.annotations)}")

for annotation in last_msg.text.annotations:
    print(annotation["text"], annotation["url_citation"]["url"])

Number of annotation(s): 7
【4:0†source】 https://ai-search-abutneva687267079310.search.windows.net/indexes/azureportal-unipd/docs/e5f642b9beb1_aHR0cHM6Ly9zdGFpZm91bmRyeXU2ODcyNjcwNzkzMTAuYmxvYi5jb3JlLndpbmRvd3MubmV0L3JlZ29sYW1lbnRpLXVuaS1wYWRvdmEvUElBTyUyMDIwMjUtMjAyNy5wZGY1_pages_102?api-version=2024-07-01&$select=chunk_id,chunk,title
【4:0†source】 https://ai-search-abutneva687267079310.search.windows.net/indexes/azureportal-unipd/docs/e5f642b9beb1_aHR0cHM6Ly9zdGFpZm91bmRyeXU2ODcyNjcwNzkzMTAuYmxvYi5jb3JlLndpbmRvd3MubmV0L3JlZ29sYW1lbnRpLXVuaS1wYWRvdmEvUElBTyUyMDIwMjUtMjAyNy5wZGY1_pages_102?api-version=2024-07-01&$select=chunk_id,chunk,title
【4:0†source】 https://ai-search-abutneva687267079310.search.windows.net/indexes/azureportal-unipd/docs/e5f642b9beb1_aHR0cHM6Ly9zdGFpZm91bmRyeXU2ODcyNjcwNzkzMTAuYmxvYi5jb3JlLndpbmRvd3MubmV0L3JlZ29sYW1lbnRpLXVuaS1wYWRvdmEvUElBTyUyMDIwMjUtMjAyNy5wZGY1_pages_102?api-version=2024-07-01&$select=chunk_id,chunk,title
【4:0†source】 https://ai-search-abutneva6872

# Run Steps

In [15]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)

print(f'Nr of run step(s): {len(run_steps["data"])}\n')
i=0
for rs in run_steps["data"]:
    i += 1
    print(f"Run step {i}: {rs}", '\n')

Nr of run step(s): 2

Run step 1: {'id': 'step_TZ1ItSBAKuf1EHRMF2vSVL2u', 'object': 'thread.run.step', 'created_at': 1744209647, 'run_id': 'run_uL1Jp2WrHJQd5J5QrCzxSauc', 'assistant_id': 'asst_1R6QzPPsYNBjaVRCdqZPWBdK', 'thread_id': 'thread_FSqsMJf59rfaWB4r67lXmZRO', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1744209652, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_BfiTthAKay3Pa7WXBxK5HlIS'}}, 'usage': {'prompt_tokens': 3399, 'completion_tokens': 339, 'total_tokens': 3738, 'prompt_token_details': {'cached_tokens': 0}}} 

Run step 2: {'id': 'step_axL4GlDi5Hz2GTLSwfJLEOTj', 'object': 'thread.run.step', 'created_at': 1744209646, 'run_id': 'run_uL1Jp2WrHJQd5J5QrCzxSauc', 'assistant_id': 'asst_1R6QzPPsYNBjaVRCdqZPWBdK', 'thread_id': 'thread_FSqsMJf59rfaWB4r67lXmZRO', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_at':

# START Teardown

In [13]:
print(f"deleting trhead: {thread}...")
project_client.agents.delete_thread(thread.id)

deleting trhead: {'id': 'thread_vruTaHnwYLksA3lQZ89CeXmh', 'object': 'thread', 'created_at': 1738151603, 'metadata': {}, 'tool_resources': {}}...


{'id': 'thread_vruTaHnwYLksA3lQZ89CeXmh', 'object': 'thread.deleted', 'deleted': True}

In [14]:
# Delete all agents

print(f"{len(project_client.agents.list_agents()['data'])} agent(s) will now be deleted")

i=0
for pca in project_client.agents.list_agents()['data']:
    i += 1
    project_client.agents.delete_agent(pca.id)
    print(f"\n{i} - Agent {pca.name} has been deleted")

1 agent(s) will now be deleted

1 - Agent aisearch-agent has been deleted


# HIC SUNT LEONES